In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.python.client import device_lib
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow
import numpy as np

In [2]:
f = open('HerMajestySpeechesDataset/train.txt', 'r', encoding='utf-8')
# text = f.read()
texts = f.readlines()
f.close()

In [3]:
len(texts)

2361

In [3]:
tokenizer = Tokenizer(oov_token='<unk>')
tokenizer.fit_on_texts(texts) 

texts2ids = tokenizer.texts_to_sequences(texts)
print ("Texts as IDs:", texts2ids[:10])
ids2texts = tokenizer.sequences_to_texts(texts2ids)
print ("IDs back to texts:", ids2texts[:10])

Texts as IDs: [[2, 181, 999, 5, 39, 584, 41, 2094, 2095, 21, 1165, 530], [21, 9, 163, 246, 8, 1000, 25, 424, 1166, 5, 18, 20], [68, 195, 781], [2, 355, 3, 7, 1347, 722, 53, 585, 40, 213, 2, 297, 3, 531, 208, 1001], [2, 181, 68, 195, 6, 781, 2096, 21, 532, 117, 872, 4, 247], [2, 195, 49, 2097, 6, 2951, 1167, 6, 151, 224, 196, 491, 533, 2098, 197, 21, 7, 2952, 2953], [8, 13, 1168, 205, 873, 5, 114, 10, 455, 187, 298, 586, 2, 259, 11, 111, 205], [58, 3, 2, 455, 10, 29, 50, 649, 20, 10, 79, 11, 66, 12, 2, 312, 3, 68], [17, 271, 7, 59, 83, 8, 782, 5, 456, 2099, 2, 2954, 3, 2, 356, 4, 214, 10, 874, 49, 492, 5, 237, 129, 4, 493, 5, 7, 875, 42], [2, 355, 3, 783, 2100, 2955, 16, 457, 5, 20, 43, 492, 248, 4, 91, 2, 69, 2956, 11, 141]]
IDs back to texts: ["the queen's messages to those celebrating their 90th birthdays on 21 april", 'on this shared occasion i send my warm congratulations to you all', 'christmas broadcast 2006', 'the birth of a baby brings great happiness but then the business of g

In [52]:
def get_ngrams(frase, size=2):
    for i in range(size):
        frase = [0] + frase
    ngrams_list = []
    for i in range(len(frase)-size):
        ngrams_list.append((tuple(frase[i:i+size]),frase[i+size]))
    return ngrams_list

all_ngrams = []
for i in texts2ids:
    all_ngrams += get_ngrams(i)

In [7]:
def co_table(lista_ocurrencias):
    table = {}
    for i in lista_ocurrencias:
        if i[0] in table:
            if i[1] in table[i[0]]:
                table[i[0]][i[1]] += 1
            else:
                table[i[0]][i[1]] = 1
        else:
            table[i[0]] = {}
            table[i[0]][i[1]] = 1
    return table

table = co_table(all_ngrams)

In [9]:
def generate_toyLM_ngram_a(table, context='aleatorio', n=15):
    if context == 'aleatorio':
        tmp = list(table.keys())
        context = tmp[np.random.randint(len(tmp))]

    cadena = list(context)
    for _ in range(n-len(context)):
        context = tuple(cadena[-len(context):])
        if context not in table.keys():
            return tokenizer.sequences_to_texts([cadena])
        else:
            new = max(table[context], key=table[context].get)
            cadena.append(new)
    return tokenizer.sequences_to_texts([cadena])

generate_toyLM_ngram_a(table)


In [11]:
def generate_toyLM_ngram_b(table, context='aleatorio', n=15):
    if context == 'aleatorio':
        tmp = list(table.keys())
        context = tmp[np.random.randint(len(tmp))]

    cadena = list(context)
    for i in range(n-len(context)):
        context = tuple(cadena[-len(context):])
        if context not in table.keys():
            return tokenizer.sequences_to_texts([cadena])
        else:
            lista_tmp = list()
            for i in table[context]:
                for j in range(table[context][i]):
                    lista_tmp.append(i)
            new = np.random.randint(len(lista_tmp))
            cadena.append(lista_tmp[new])
    return tokenizer.sequences_to_texts([cadena])

generate_toyLM_ngram_b(table, (424, 1166))

# toyLM_LSTM

In [13]:
def train_generate(text, max_seq_length=10):
    train_set = dict()
    for frase in text:
        for word_index in range(len(frase)):
            if word_index < max_seq_length:
                train_set[tuple(pad_sequences([frase[:word_index]], maxlen=max_seq_length)[0])] = frase[word_index]
    return train_set
    
train_set = train_generate(texts2ids)

In [46]:
x_train = np.array(list(train_set.keys()))
y_train = to_categorical(list(train_set.values()), num_classes=len(tokenizer.word_index))

In [51]:
model = Sequential([
    Embedding(5614, 20, input_length=10),
    LSTM(64),
    Dense(5614, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(x_train, y_train, epochs=10)

Epoch 1/10
502/502 [==============================] - 51s 10ms/step - loss: 6.8330 - accuracy: 0.0585
Epoch 2/10
502/502 [==============================] - 5s 11ms/step - loss: 6.2733 - accuracy: 0.0639
Epoch 3/10
502/502 [==============================] - 5s 10ms/step - loss: 6.1494 - accuracy: 0.0814
Epoch 4/10
502/502 [==============================] - 5s 10ms/step - loss: 6.0370 - accuracy: 0.0864
Epoch 5/10
502/502 [==============================] - 5s 10ms/step - loss: 5.9362 - accuracy: 0.0950
Epoch 6/10
502/502 [==============================] - 5s 10ms/step - loss: 5.8380 - accuracy: 0.0984
Epoch 7/10
502/502 [==============================] - 5s 10ms/step - loss: 5.7258 - accuracy: 0.1048
Epoch 8/10
502/502 [==============================] - 5s 9ms/step - loss: 5.6050 - accuracy: 0.1146
Epoch 9/10
502/502 [==============================] - 5s 10ms/step - loss: 5.4871 - accuracy: 0.1208
Epoch 10/10
502/502 [==============================] - 5s 10ms/step - loss: 5.3725 - accura